In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Ex.1 - Stokes problem - patch test

\begin{equation*}
\begin{cases}
- \Delta \boldsymbol{u} + \nabla  p  = \boldsymbol{0} & {\rm in} \ \Omega=(0,3)\times(0,1), \\
{\rm div}\,\boldsymbol{u} = 0 & {\rm in} \ \Omega, \\
(\nabla\boldsymbol{u}-pI)\boldsymbol{n} = \boldsymbol{0} & {\rm on} \ \Gamma_1\cup\Gamma_2,\\
\boldsymbol{u} = \boldsymbol{0} & {\rm on} \ \Gamma_3, \\
\boldsymbol{u} = \boldsymbol{g}_\text{D} & {\rm on} \ \Gamma_4.
\end{cases}
\end{equation*}

with $\boldsymbol{g}_\text{D} = 1\boldsymbol{i}$.

In [ ]:
from firedrake import RectangleMesh

# Build the mesh
n = 10
mesh = RectangleMesh(3*n, n, 3, 1)

fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend()

In [ ]:
# Function spaces

# P1-P0
V = VectorFunctionSpace(mesh, 'P', 1)
Q = FunctionSpace(mesh, 'DP', 0) # NB: P0 are DISCONTINUOUS elements (DP)

# P1-P1
# V = VectorFunctionSpace(mesh, 'P', 1)
# Q = FunctionSpace(mesh, 'P', 1)

# P1b-P1
# The enrichment of the velocity space has to be done at the finite element level
V1_el = FiniteElement('CG', mesh.ufl_cell(), 1)
B_el = FiniteElement('Bubble', mesh.ufl_cell(), mesh.topological_dimension() + 1)
V_el = VectorElement(NodalEnrichedElement(V1_el, B_el))
V = FunctionSpace(mesh, V_el)
Q = FunctionSpace(mesh, 'P', 1)

# P2-P1
# V = VectorFunctionSpace(mesh, 'P', 2)
# Q = FunctionSpace(mesh, 'P', 1)

W = MixedFunctionSpace([V, Q])
print('Ndofs - velocity :',V.dim(),', pressure :',Q.dim(),', total :',W.dim())

# Finite element functions
u, p = TrialFunctions(W)
v, q = TestFunctions(W)

In [ ]:
# Boundary conditions (strong)
bc3 = DirichletBC(W.sub(0), Constant((0., 0.)), 3)
bc4 = DirichletBC(W.sub(0), Constant((1., 0.)), 4)
bcs = (bc3, bc4)

# Variational formulation
a = inner(grad(u), grad(v)) * dx - div(v) * p * dx + q * div(u) * dx
L = inner(Constant((0.0,0.0)), v) * dx
  # Dummy rhs (=0) to ensure that the solve recognize a==L as a linear problem

# Solution (NB: do not use the same name u,v,p,q of the trial/test functions)
wh = Function(W)
solve(a == L, wh, bcs=bcs)
uh, ph = wh.subfunctions

In [ ]:
# Variational formulation (penalty method)
eps = 1.e-30
a = inner(grad(u), grad(v)) * dx - div(v) * p * dx + q * div(u) * dx \
    + 1./eps*inner(u,v)*ds(3) + 1./eps*inner(u,v)*ds(4)
L = 1./eps*inner(Constant((1.,0.)),v) * ds(4)

# Solution (NB: do not use the same name u,v,p,q of the trial/test functions)
wh = Function(W)
solve(a == L, wh, bcs=bcs)
uh, ph = wh.subfunctions

In [ ]:
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')

In [ ]:
# Error computation
x = SpatialCoordinate(mesh)
u_ex = as_vector([x[1],0.])
grad_u_ex = as_tensor([[0.,1.],[0.,0.]])
p_ex = Constant(0.)
errL2u = sqrt(assemble( inner(uh-u_ex,uh-u_ex) * dx ))
errH10u = sqrt(assemble( inner(grad(uh)-grad_u_ex,grad(uh)-grad_u_ex) * dx ))
errL2p = sqrt(assemble( inner(ph-p_ex,ph-p_ex) * dx ))
print('Errors - L2-u:', errL2u, ', H10-u:', errH10u, ', L2-p:', errL2p)